![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [125]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [126]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

1. Data Analysis Exploratory 

In [127]:
#index, columns, null & type
print(ds_jobs_transformed.info())

#nombre de valeur unique dans une colonne 
print(ds_jobs_transformed.nunique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

2. Intergers, Floats and Unordered categories

In [128]:

ds_jobs_transformed['relevant_experience']=ds_jobs_transformed['relevant_experience'].map({'Has relevant experience':1, 'No relevant experience':0})
col_cat = ['gender', 'enrolled_university', 'education_level', 'major_discipline', 'experience',                  'company_size', 'company_type','last_new_job', 'city']
col_int32 = ['training_hours', 'student_id']
col_float16 = ['city_development_index', 'job_change']
col_bool=['relevant_experience', 'job_change']


ds_jobs_transformed[col_cat] = ds_jobs_transformed[col_cat].astype('category')
ds_jobs_transformed[col_int32] = ds_jobs_transformed[col_int32].astype('int32')
ds_jobs_transformed[col_float16] = ds_jobs_transformed[col_float16].astype('float16')
ds_jobs_transformed[col_bool] = ds_jobs_transformed[col_bool].astype('bool')

print(ds_jobs_transformed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

3. Ordered Categories

In [129]:
ordered_cat = ['education_level', 'experience', 'company_size','last_new_job']
print(ds_jobs_transformed[ordered_cat].apply(lambda x: x.unique()))

# Creating an ordered list

experience_levels = ['<1'] + list(map(str, range(1, 21))) + ['>20']
job_lasting = ['never'] + list(map(str, range(1, 5))) + ['>4']

# Creating ordered categorical & data types


print(ds_jobs_transformed['enrolled_university'].cat.categories)
enrolled = pd.CategoricalDtype(categories=['no_enrollment', 'Part time course', 'Full time course'], ordered=True) 

print(ds_jobs_transformed['education_level'].cat.categories)
educat = pd.CategoricalDtype(categories=['Primary School','High School', 'Graduate', 'Masters','Phd'], ordered=True) 

print(ds_jobs_transformed['company_size'].cat.categories)
compan_s = pd.CategoricalDtype(categories=['<10','10-49','50-99', '100-499', '500-999', '1000-4999','5000-9999', '10000+'], ordered=True)

print(ds_jobs_transformed['experience'].cat.categories)
exper = pd.CategoricalDtype(categories = experience_levels, ordered=True)

print(ds_jobs_transformed['last_new_job'].cat.categories)
last_job =pd.CategoricalDtype(categories = job_lasting, ordered=True)


#converting columns


ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype(enrolled)
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype(educat)
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype(exper)
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype(compan_s)
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype(last_job)

education_level    ['Graduate', 'Masters', 'High School', NaN, 'P...
experience         ['>20', '15', '5', '<1', '11', ..., '6', '9', ...
company_size       [NaN, '50-99', '<10', '10000+', '5000-9999', '...
last_new_job       ['1', '>4', 'never', '4', '3', '2', NaN]
Categ...
dtype: object
Index(['Full time course', 'Part time course', 'no_enrollment'], dtype='object')
Index(['Graduate', 'High School', 'Masters', 'Phd', 'Primary School'], dtype='object')
Index(['10-49', '100-499', '1000-4999', '10000+', '50-99', '500-999',
       '5000-9999', '<10'],
      dtype='object')
Index(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2',
       '20', '3', '4', '5', '6', '7', '8', '9', '<1', '>20'],
      dtype='object')
Index(['1', '2', '3', '4', '>4', 'never'], dtype='object')


4. Filtering Ordered Categorical Columns

In [130]:
#Category codes
codes_exp = ds_jobs_transformed['experience'].cat.codes
codes_size = ds_jobs_transformed['company_size'].cat.codes

# Filter the values
filter_exp = (codes_exp>9)
filter_size = (codes_size>4)

#DataFrame with the value
ds_jobs_transformed = ds_jobs_transformed.loc[filter_exp & filter_size]